- Building a CNN from scratch using keras for emotion detection using our live camera feed

# imports

In [25]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, GlobalAveragePooling2D, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os

In [6]:
# setting up some convenience variables
num_classes = 5
img_rows, img_cols = 48,48
batch_size = 32

In [7]:
# using the fer2013 dataset from kaggle
train_data_dir = "train"
validation_data_dir = "validation"

In [9]:
# augmenting our dataset
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    shear_range = 0.3,
    zoom_range = 0.3,
    width_shift_range = 0.4,
    height_shift_range = 0.4,
    horizontal_flip = True,
    fill_mode = "nearest"
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
# rescaling the validation data and checking the model with raw data different from training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode = "grayscale",
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode = "categorical",
    shuffle = True)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode = "grayscale",
    target_size = (img_rows, img_cols),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

Found 24256 images belonging to 5 classes.
Found 3006 images belonging to 5 classes.


In [12]:
# initializing our model
model = Sequential()

In [13]:
# 1st block
model.add(Conv2D(32,(3,3), padding='same', kernel_initializer="he_normal", input_shape=(img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_initializer='he_normal', input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [15]:
# 2nd block
model.add(Conv2D(64,(3,3), padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [16]:
# 3rd block
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3), padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [17]:
# 4th block
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3), padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [20]:
# 5th block
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [21]:
# 6th block
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [22]:
# 7th block
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [23]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

## compiling and training the model

In [26]:
checkpoint = ModelCheckpoint('Emotion_Detection.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=3,
                         verbose=1,
                         restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.2,
                             patience=3,
                             verbose=1,
                             min_delta=0.0001)

callbacks = [earlystop, checkpoint, reduce_lr]

In [27]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.001),
             metrics=['accuracy'])

In [30]:
# convenience variables
train_samples = 24176
val_samples = 3006
epochs = 25

In [31]:
# finally training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_samples//batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=val_samples//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
755/755 [==============================] - ETA: 0s - loss: 1.8374 - accuracy: 0.2413
Epoch 00001: val_loss improved from inf to 1.54973, saving model to Emotion_Detection.h5
755/755 [==============================] - 228s 301ms/step - loss: 1.8374 - accuracy: 0.2413 - val_loss: 1.5497 - val_accuracy: 0.2907
Epoch 2/25
755/755 [==============================] - ETA: 0s - loss: 1.5740 - accuracy: 0.2834
Epoch 00002: val_loss did not improve from 1.54973
755/755 [==============================] - 238s 315ms/step - loss: 1.5740 - accuracy: 0.2834 - val_loss: 1.5849 - val_accuracy: 0.2665
Epoch 3/25
755/755 [==============================] - ETA: 0s - loss: 1.5481 - accuracy: 0.3017
Epoch 00003: val_loss improved from 1.54973 to 1.52247, saving model to Emotion_Detection.h5
755/755 [==============================] - 236s 312ms/step - loss: 1.5481 - accuracy: 0.3017 - val_loss: 1.5225 - val_accuracy: 0.313